### Notebook to run the complete eval of our method for Handmade questions

https://docs.google.com/spreadsheets/d/1xgkQt_1NiwXEZhZUvTypgecG5wMCl8_brc6w1bhaOtE/edit?gid=0#gid=0



In [1]:
# Optional : rm -r storage to remove storage


## Step 0 : download videos

In [2]:
import pandas as pd

df_questions = pd.read_csv('Handmade_questions.csv')
print(df_questions.keys())
df_questions = df_questions.dropna(subset=['Question']) #drop any rows that aren't filled yet
df_questions

Index(['VideoID', 'Question', 'Answer1 (corr)', 'Answer2', 'Answer3',
       'Answer4', 'youtubeURL', 'Unnamed: 7', 'Unnamed: 8',
       'https://www.youtube.com/watch?v='],
      dtype='object')


,VideoID,Question,Answer1 (corr),Answer2,Answer3,Answer4,youtubeURL,Unnamed: 7,Unnamed: 8,https://www.youtube.com/watch?v=
0,zwbCD3CUFIs,What color is the spatula on the cutting board?,Green,Red,Blue,Black,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
1,zwbCD3CUFIs,What green ingredient is added to the pan ?,Spring onions,Green pepper,Green beans,Cucumber,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
2,zwbCD3CUFIs,What is the purpose of the large blue object o...,To provide heat to cook,It is a trash can,It is a pan,It's a decoration,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
3,zwbCD3CUFIs,How many prawns are cooking in the pan?,approximately 25,About 4 big prawns,approximately 80 prawns,there are no prawns,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
5,XzgR14o6b80,What is the color if the liquid the two steaks...,Yellow-brown,Red,Green,Transparent,https://www.youtube.com/watch?v=XzgR14o6b80,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
73,i8BjqgemuXo,What distinguishes this dish?,Honey ham and parma ham,Bread type,Replace with turkey,Serve extra cold,https://www.youtube.com/watch?v=i8BjqgemuXo,NaN,NaN,NaN
75,g1jFJYtQ9_s,What flour is used traditionally for this dish?,Whole wheat flour,Cornmeal,Corn flour,Rice flour,https://www.youtube.com/watch?v=g1jFJYtQ9_s,NaN,NaN,NaN
76,g1jFJYtQ9_s,How long do you rest the dish prior to rolling?,30 minutes,20 minutes,10 minutes,45 minutes,https://www.youtube.com/watch?v=g1jFJYtQ9_s,NaN,NaN,NaN
77,g1jFJYtQ9_s,How to make the dish fluffy?,Open flame placement,Flattening with spatula,Add yeast to leaven,Freeze prior to cooking,https://www.youtube.com/watch?v=g1jFJYtQ9_s,NaN,NaN,NaN


In [3]:
ALL_URLS = df_questions['VideoID'].unique().tolist()

In [4]:
#################################################################################################################
WANT_TO_RUN_DL = True
#################################################################################################################



from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

base_path = './storage'

if WANT_TO_RUN_DL:

    for URL in ALL_URLS:

        os.makedirs(f"{base_path}/{URL}", exist_ok=True)
        url = f"https://www.youtube.com/watch?v={URL}"

        yt = YouTube(url, on_progress_callback = on_progress)
        print(yt.title)
        ys = yt.streams.get_highest_resolution()
        ys.download(f"{base_path}/{URL}", filename='video')

        caption = yt.captions.get_by_language_code('a.en')

        #save captions to ./
        caption.save_captions(f"{base_path}/{URL}/caption.srt")

Singapore-Style Curry Fried Rice Noodles | Asian Bites


/scratch/1034370/ipykernel_1433680/1625065730.py:25: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')
/scratch/1034370/ipykernel_1433680/1625065730.py:25: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Experience Culinary Bliss with Gordon Ramsay Beef Tacos with Wasabi Mayo


## Step 1 : Generate the CSV from input videos

In [5]:
import cv2
import os
import csv
import pysrt


csv_output = "image_transcript_mapping.csv"

def video_to_images_with_transcript(video_path, output_folder, srt_path, csv_output, frame_interval=1):
    """
    Extract video frames as images, and create csv in format of [index, image_path, transcript]

    video_path: Path to the input video file
    output_folder: Directory to save the image frames
    srt_path: Path to the SRT subtitle file
    csv_output: Path to the output CSV file mapping images to transcript text
    frame_interval: Interval of frames to save
    """
    video = cv2.VideoCapture(video_path)
    
    if not video.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    os.makedirs(output_folder, exist_ok=True)
    
    subs = pysrt.open(srt_path)
    
    fps = video.get(cv2.CAP_PROP_FPS)
    
    frame_count = 0
    saved_count = 0
    csv_data = []

    while True:
        success, frame = video.read()
        #mkdir f"{output_folder}/frames" with os
        os.makedirs(f"{output_folder}/frames", exist_ok=True)
        
        if not success:
            break

        if frame_count % frame_interval == 0:
            # Save the frame as image
            img_filename = f"{output_folder}/frames/frame_{frame_count:06d}.jpg"
            print(f"Saving frame {frame_count} to {img_filename}")
            cv2.imwrite(img_filename, frame)
            saved_count += 1

            # Map subtitle with timestamp

            timestamp = frame_count / fps

            transcript_text = ""
            for sub in subs:
                start_time = sub.start.ordinal / 1000
                end_time = sub.end.ordinal / 1000
                if abs(start_time - timestamp) < 11:
                    transcript_text += sub.text+" "
                    
            #make the img_filename the full path
            curr_path = os.path.join(os.getcwd(), img_filename)
            print(curr_path)

            if transcript_text == "":
                transcript_text = "No transcript available"
            csv_data.append([int(frame_count/frame_interval), curr_path, transcript_text,timestamp])

        frame_count += 1

    video.release()

    with open(f'{output_folder}/{csv_output}', mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['index', 'image_path', 'transcript_text','timestamp'])
        writer.writerows(csv_data)

    print(f"Saved {saved_count} frames from {video_path} to {output_folder}")
    print(f"Transcript mapping saved to {csv_output}")






In [6]:

#################################################################################################################
WANT_TO_RUN_CSV_CREATION = True
INTERVAL_TIME = 90
#################################################################################################################

import os
import multiprocessing

def process_url(URL):
    """
    Process a single URL by extracting frames and generating CSV
    
    :param URL: The URL/path to process
    """
    video_path = f"{base_path}/{URL}/video.mp4"
    srt_path = f"{base_path}/{URL}/caption.srt"
    
    # Launch the script to extract frames and generate CSV
    video_to_images_with_transcript(video_path, f"{base_path}/{URL}", srt_path, csv_output, INTERVAL_TIME)

def main():
    # Determine the number of processes (4 as you specified, or use number of CPU cores)
    num_processes = 4
    
    # Use multiprocessing Pool to process URLs in parallel
    with multiprocessing.Pool(processes=num_processes) as pool:
        # Map the process_url function to ALL_URLS
        pool.map(process_url, ALL_URLS)

if __name__ == '__main__':
    if WANT_TO_RUN_CSV_CREATION:
        main()

Saving frame 0 to ./storage/zwbCD3CUFIs/frames/frame_000000.jpgSaving frame 0 to ./storage/XzgR14o6b80/frames/frame_000000.jpg

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000000.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_000000.jpg
Saving frame 90 to ./storage/zwbCD3CUFIs/frames/frame_000090.jpgSaving frame 90 to ./storage/XzgR14o6b80/frames/frame_000090.jpg

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_000090.jpg/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000090.jpg

Saving frame 180 to ./storage/zwbCD3CUFIs/frames/frame_000180.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000180.jpg
Saving frame 180 to ./storage/XzgR14o6b80

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_002070.jpg
Saving frame 1710 to ./storage/XzgR14o6b80/frames/frame_001710.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_001710.jpg
Saving frame 2160 to ./storage/zwbCD3CUFIs/frames/frame_002160.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_002160.jpg
Saving frame 2250 to ./storage/zwbCD3CUFIs/frames/frame_002250.jpg
Saving frame 1800 to ./storage/XzgR14o6b80/frames/frame_001800.jpg/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_002250.jpg

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_001800.jpg
Saving frame 2340 to ./storage/zwbCD3CUFIs/frames/frame_002340.jpg
/storage/ice1/0/4/bpopper3/aaa

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_003330.jpg
Saving frame 4230 to ./storage/zwbCD3CUFIs/frames/frame_004230.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_004230.jpg
Saving frame 3420 to ./storage/XzgR14o6b80/frames/frame_003420.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_003420.jpg
Saving frame 4320 to ./storage/zwbCD3CUFIs/frames/frame_004320.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_004320.jpg
Saving frame 4410 to ./storage/zwbCD3CUFIs/frames/frame_004410.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_004410.jpg
Saving frame 3510 to ./storage/XzgR14o6b80/frames/frame_003510.jpg
/storage/ice1/0/4/bpopper3/aaa

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_005040.jpg
Saving frame 6300 to ./storage/zwbCD3CUFIs/frames/frame_006300.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_006300.jpg
Saving frame 5130 to ./storage/XzgR14o6b80/frames/frame_005130.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_005130.jpg
Saving frame 6390 to ./storage/zwbCD3CUFIs/frames/frame_006390.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_006390.jpg
Saving frame 5220 to ./storage/XzgR14o6b80/frames/frame_005220.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_005220.jpg
Saving frame 6480 to ./storage/zwbCD3CUFIs/frames/frame_006480.jpg
/storage/ice1/0/4/bpopper3/aaa

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_006840.jpg
Saving frame 8280 to ./storage/zwbCD3CUFIs/frames/frame_008280.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_008280.jpg
Saving frame 6930 to ./storage/XzgR14o6b80/frames/frame_006930.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_006930.jpg
Saving frame 8370 to ./storage/zwbCD3CUFIs/frames/frame_008370.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_008370.jpg
Saving frame 7020 to ./storage/XzgR14o6b80/frames/frame_007020.jpg
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_007020.jpg
Saving frame 8460 to ./storage/zwbCD3CUFIs/frames/frame_008460.jpg
/storage/ice1/0/4/bpopper3/aaa

## Step 2 : Generate FAISS embeddings from the CSVs

In [7]:

#################################################################################################################
WANT_TO_CREATE_FAISS = True
#Takes about 15 sec per video on a good GPU (depends also how many frames per vid, this is for interval 60)
DEVICE = "cuda:0" #only runs fast on GPU

HF_CACHE = "/home/hice1/bpopper3/scratch/hug_face_files/hub_cache"
#################################################################################################################


import json
import os
import numpy as np
from numpy.linalg import norm
import cv2
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from termcolor import cprint
import faiss
import pandas as pd

BT_processor = BridgeTowerProcessor.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=HF_CACHE) #.to(DEVICE)
BT_model = BridgeTowerForContrastiveLearning.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=HF_CACHE).to(DEVICE)


def encode_single_image_text(image_path, text):
    print(image_path)
    print(text)
    image = Image.open(image_path)
    encoding = BT_processor(image, text, return_tensors="pt").to(DEVICE)
    outputs = BT_model(**encoding)
    return outputs['cross_embeds'].cpu().detach().numpy()




if WANT_TO_CREATE_FAISS:
    for URL in ALL_URLS:
        input_folder = f"{base_path}/{URL}"
        
        
            #load ./paired_image_data.csv
        df_mapping = pd.read_csv(f'{input_folder}/image_transcript_mapping.csv')
        #rows :             index,image_path,transcript_text,timestamp

        #create empty FAISS index
        

        # Create embeddings for each row
        embeddings = np.zeros((len(df_mapping), 512))
        for _, row in tqdm(df_mapping.iterrows()):


            embedding = encode_single_image_text(row['image_path'], row['transcript_text'])
            #reshape to (512,)
            embedding = embedding.reshape(512)
            embeddings[row['index'],:] = embedding

            print(embedding.shape)

        # Initialize FAISS index
        dimension = 512  # dimension of your embeddings
        index = faiss.IndexFlatL2(dimension)

        # Add embeddings to the FAISS index
        index.add(embeddings)

        # Create a mapping between FAISS ids and dataframe indices
        id_to_index = {i: idx for i, idx in enumerate(df_mapping.index)}

        # Save the FAISS index
        faiss.write_index(index, f"{input_folder}/faiss_database.bin")
        print('saved faiss index')

/home/hice1/bpopper3/.conda/envs/vlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hice1/bpopper3/.conda/envs/vlm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BridgeTowerForContrastiveLearning were not initialized from the model checkpoint at BridgeTower/bridgetower-large-itm-mlm-itc and are newly initialized: ['logit_scale']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
0it [00:00, ?it/

/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000000.jpg
hey guys I'm Amy and I'm Julie we are the Duncan sisters welcome to the scoffs kitchen today in our Asian bite series we are doing a Singapore fried rice noodle with curry is you can see it's this bright yellow sort of concoction 


3it [00:08,  2.17s/it]

(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000090.jpg
hey guys I'm Amy and I'm Julie we are the Duncan sisters welcome to the scoffs kitchen today in our Asian bite series we are doing a Singapore fried rice noodle with curry is you can see it's this bright yellow sort of concoction and it's super yummy so let's get started so the notice that we're going 
(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000180.jpg
hey guys I'm Amy and I'm Julie we are the Duncan sisters welcome to the scoffs kitchen today in our Asian bite series we are doing a Singapore fried rice noodle with curry is you can see it's this bright yellow sort of concoction and it's super yummy so let's get started so the notice that we're going 
(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_0

5it [00:08,  1.10s/it]

...

(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_007290.jpg
mayonnaise inside Taco just spinkle that delicious pickled cabbage and then just start lining my taco with three or four slices touch more on my spicy Mayo that is how I'd make the perfect 
(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_007380.jpg
delicious pickled cabbage and then just start lining my taco with three or four slices touch more on my spicy Mayo that is how I'd make the perfect Tacko 
(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_007470.jpg
start lining my taco with three or four slices touch more on my spicy Mayo that is how I'd make the perfect Tacko 


86it [00:06, 12.79it/s]

(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_007560.jpg
start lining my taco with three or four slices touch more on my spicy Mayo that is how I'd make the perfect Tacko 
(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_007650.jpg
more on my spicy Mayo that is how I'd make the perfect Tacko 
(512,)
/storage/ice1/0/4/bpopper3/aaaaaaaa/LLaVA_RAG/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_007740.jpg
Mayo that is how I'd make the perfect Tacko 


87it [00:06, 12.71it/s]

(512,)
saved faiss index


## Step 3 : Chat with FAISS Db as RAG

In [8]:
import faiss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from PIL import Image
from termcolor import cprint
import pandas as pd


def encode_text_query(text):
    global index
    global df
    #we suppose BT_processor and BT_model are loaded from cell above
    #create random Image 32x32
    image = Image.fromarray(np.random.randint(0,255,(32,32,3),dtype=np.uint8))
    encoding = BT_processor(image, text, return_tensors="pt").to(DEVICE)
    outputs = BT_model(**encoding)
    #reshape to (512,)

    return outputs['text_embeds'].cpu().detach().numpy().reshape(-1)


def get_n_closest_index(embedding,n):
    global index
    global df
    D, I = index.search(np.array([embedding]), n)
    #I[0] has the indexes of the n closest images
    return I[0]

def set_index(folder_path):
    global index
    global df
    index = faiss.read_index(folder_path+'/faiss_database.bin')
    df = pd.read_csv(folder_path+'/image_transcript_mapping.csv')



In [9]:
def return_image_and_enhanced_query(question,answer_a,answer_b,answer_c,answer_d,folder_path):
    global index
    global df
    set_index(folder_path)
    text_embedding = encode_text_query(question)
    
    #return query with 1 closest image/text pair to enhance the query

    closest_index = get_n_closest_index(text_embedding,1)
    print(f"Closest index is {closest_index[0]} for query '{question}'")
    return create_enhanced_conversation(df.iloc[closest_index[0]]['transcript_text'],question,Image.open(df.iloc[closest_index[0]]['image_path']),answer_a,answer_b,answer_c,answer_d)


def debug_top3_frames(question,answer_a,answer_b,answer_c,answer_d,folder_path):
    global index
    global df
    set_index(folder_path)
    text_embedding = encode_text_query(question)
    
    
    


    closest_index = get_n_closest_index(text_embedding,3)
    top1_text = df.iloc[closest_index[0]]['transcript_text']
    return closest_index[0],closest_index[1],closest_index[2],top1_text
    

def create_enhanced_conversation(textt,question,image,answer_a,answer_b,answer_c,answer_d):
    global index
    global df
    
    return [{
        "role": "user",
        "content": [
            {"type": "text", "text" : f"""Please answer this question : 'What is 2+2'
            This transcript can also help you answer : 'Addition is a mathematical commutative operation that' .
            It is possible that the image helps you answer the question, but it is possible that it does not
            Answer A: 4
            Answer B: 6
            Answer C: 9
            Answer D: 12"""}
        ],
    },
    {
        "role": "assistant",
        "content": [
   
            {"type": "text", "text" : "A"}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text" : f"""Please answer this question : '{question}'
            This transcript can also help you answer : '{textt}' .
            It is possible that the image helps you answer the question, but it is possible that it does not
            Answer A: {answer_a}
            Answer B: {answer_b}
            Answer C: {answer_c}
            Answer D: {answer_d}"""}
        ],
    }],image

In [10]:
#This script loads LLaVA so it's on a big GPU
import json
import os
import numpy as np
from numpy.linalg import norm
import cv2
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from termcolor import cprint


########################## LOADING LLAVA ##########################


print("beginning to load LLaVA")

llava_processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", device=DEVICE,cache_dir=HF_CACHE)


from transformers import LlavaNextForConditionalGeneration, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,  # Threshold for weight offloading
    llm_int8_skip_modules=["lm_head"],  # Prevent quantization of specific layers
    llm_int8_has_fp16_weights=True  # Mixed precision with fp16 weights
)

llava_model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", quantization_config=quantization_config, device_map=DEVICE,cache_dir=HF_CACHE)


beginning to load LLaVA


Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
Unused kwargs: ['llm_int8_has_fp16_weights']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 4/4 [00:26<00:00,  6.51s/it]


In [11]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

# Initialize an empty list to store results
results = []

total_correct = 0
for index, row in df_questions.iterrows():
    URL = row['VideoID']
    question = row['Question']
    correct_answer = row['Answer1 (corr)']
    wrong_answer1 = row['Answer2']
    wrong_answer2 = row['Answer3']
    wrong_answer3 = row['Answer4']
    
    # Shuffle and assign answer_a, b, c, d
    answers = [correct_answer, wrong_answer1, wrong_answer2, wrong_answer3]
    np.random.shuffle(answers)
    answer_a = answers[0]
    answer_b = answers[1]
    answer_c = answers[2]
    answer_d = answers[3]
    
    folder_path = f"{base_path}/{URL}"
    
    conv, image = return_image_and_enhanced_query(question, answer_a, answer_b, answer_c, answer_d, folder_path)
    top1, top2, top3,top1text = debug_top3_frames(question, answer_a, answer_b, answer_c, answer_d, folder_path)
    
    # Paths for top frames
    path_top1 = f"{folder_path}/frames/frame_{(INTERVAL_TIME*top1):06d}.jpg"
    path_top2 = f"{folder_path}/frames/frame_{(INTERVAL_TIME*top2):06d}.jpg"
    path_top3 = f"{folder_path}/frames/frame_{(INTERVAL_TIME*top3):06d}.jpg"
    
    prompt = llava_processor.apply_chat_template(conv, add_generation_prompt=True)
    inputs = llava_processor(image, prompt, return_tensors="pt").to(DEVICE)
    
    with torch.no_grad():
        # Get the model's output logits directly
        model_outputs = llava_model(**inputs)
        logits = model_outputs.logits[0, -1, :]
        
        # Apply softmax to get probabilities
        probabilities = F.softmax(logits, dim=0)
        
        answer_tokens = {
            'C': 334,
            'A': 330,
            'B': 365,
            'D': 384
        }
        
        # Check probabilities of known answer tokens
        answer_probs = {label: probabilities[token_id].item() 
                        for label, token_id in answer_tokens.items()}
        
        # Find the top answer among the known tokens
        top_answer = max(answer_probs, key=answer_probs.get)
        
        def top_answer_to_idx(character):
            if character == "A":
                return 0
            if character == "B":
                return 1
            if character == "C":
                return 2
            if character == "D":
                return 3
        
        top_answer_text = answers[top_answer_to_idx(top_answer)]
        
        # Determine if the selected answer is correct
        is_correct = (top_answer_text == correct_answer)
        if is_correct:
            total_correct += 1
        
        # Check if top token is not in known answers
        top_token_id = torch.argmax(probabilities).item()
        not_top_token = top_token_id not in answer_tokens.values()
        if not_top_token:
            top_token = llava_processor.tokenizer.convert_ids_to_tokens([top_token_id])[0]
        else:
            top_token = None
        
        # Prepare result dictionary
        result = {
            'VideoURL': URL,
            'Question': question,
            'Selected Answer': top_answer_text,
            'isSelectedAnswerCorrect': is_correct,
            'Choice A': answer_a,
            'Choice A Probability': answer_probs.get('A', None),
            'Choice B': answer_b,
            'Choice B Probability': answer_probs.get('B', None),
            'Choice C': answer_c,
            'Choice C Probability': answer_probs.get('C', None),
            'Choice D': answer_d,
            'Choice D Probability': answer_probs.get('D', None),
            'Correct Answer': correct_answer,
            'Not Top Token': not_top_token,
            'Top Token': top_token,
            'Chosen index for top frame': top1,
            'Second best index': top2,
            'Third best index': top3,
            'Path Top1': path_top1,
            'Path Top2': path_top2,
            'Path Top3': path_top3,
            'top 1 text' : top1text
        }
        
        results.append(result)
        
        print(f"Question: {question}")
        print(f"Selected Answer: {top_answer_text}")
        print(f"Correct: {is_correct}")
        print('----------------------------------------------------------------------')

# Create DataFrame from results
results_df = pd.DataFrame(results)

import time
t = time.time()
# Save to CSV
results_df.to_csv(f'{t}_model_answer_results.csv', index=False)

# Optional: Print total correct answers
print(f"Total Correct Answers: {total_correct}/{len(df_questions)}")
print(f"Accuracy: {total_correct/len(df_questions)*100:.2f}%")


Closest index is 43 for query 'What color is the spatula on the cutting board?'


/home/hice1/bpopper3/.conda/envs/vlm/lib/python3.11/site-packages/jinja2/lexer.py:652: DeprecationWarning: invalid escape sequence '\s'
  .decode("unicode-escape")
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Question: What color is the spatula on the cutting board?
Selected Answer: Black
Correct: False
----------------------------------------------------------------------
Closest index is 14 for query 'What green ingredient is added to the pan ?'
Question: What green ingredient is added to the pan ?
Selected Answer: Spring onions
Correct: True
----------------------------------------------------------------------
Closest index is 58 for query 'What is the purpose of the large blue object on the table ?'
Question: What is the purpose of the large blue object on the table ?
Selected Answer: To provide heat to cook
Correct: True
----------------------------------------------------------------------
Closest index is 20 for query 'How many prawns are cooking in the pan?'
Question: How many prawns are cooking in the pan?
Selected Answer: About 4 big prawns
Correct: False
----------------------------------------------------------------------
Closest index is 28 for query 'What is the color if the

RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /home/circleci/miniconda/conda-bld/faiss-pkg_1709244517602/work/faiss/impl/io.cpp:67: Error: 'f' failed: could not open ./storage/vow-kxTPatc/faiss_database.bin for reading: No such file or directory